#### You can change the port the server is running on right here :

In [1]:
PORT = 8080

In [2]:
# https://www.youtube.com/watch?v=3QiPPX-KeSc

import socket
import threading
import random
import string
import time
import pickle

HEADER = 64
SERVER = "0.0.0.0"
ADDR = (SERVER, PORT)
FORMAT = 'utf-8'
DISCONNECT_MESSAGE = ['!DISCONNECT']

server = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
server.bind(ADDR)

connections = []
rooms = []
stack = [[0,0], [1,0], [1,0], [2,0], [2,0], [3,0], [3,0], [4,0], [4,0], [5,0], [5,0], [6,0], [6,0], [7,0], [7,0], [8,0], [8,0], [9,0], [9,0], [10,0], [10,0], [11,0], [11,0], [12,0], [12,0], 
             [0,1], [1,1], [1,1], [2,1], [2,1], [3,1], [3,1], [4,1], [4,1], [5,1], [5,1], [6,1], [6,1], [7,1], [7,1], [8,1], [8,1], [9,1], [9,1], [10,1], [10,1], [11,1], [11,1], [12,1], [12,1], 
             [0,2], [1,2], [1,2], [2,2], [2,2], [3,2], [3,2], [4,2], [4,2], [5,2], [5,2], [6,2], [6,2], [7,2], [7,2], [8,2], [8,2], [9,2], [9,2], [10,2], [10,2], [11,2], [11,2], [12,2], [12,2], 
             [0,3], [1,3], [1,3], [2,3], [2,3], [3,3], [3,3], [4,3], [4,3], [5,3], [5,3], [6,3], [6,3], [7,3], [7,3], [8,3], [8,3], [9,3], [9,3], [10,3], [10,3], [11,3], [11,3], [12,3], [12,3], 
             [0,4], [0,4], [0,4], [0,4], [5,4], [5,4], [5,4], [5,4]]
stackByRoom = []
heapByRoom = []
wayByRoom = []

In [3]:
def handle_client(conn, addr):
    global connections
    global rooms
    global stackByRoom
    global heapByRoom
    global stack
    connections.append(conn)
    print(f"[INFO] New connection, {addr} connected.")
    connected = True
    while connected:
        msg_lenght = conn.recv(HEADER).decode(FORMAT)
        if msg_lenght:
            msg_lenght = int(msg_lenght)
            msg = pickle.loads(conn.recv(msg_lenght))
            if msg == DISCONNECT_MESSAGE:
                connected = False
                print(f"[INFO] A client disconnected")
                print(f"[ACTIVE CONNECTION] {threading.active_count() - 9 }")
                currentRoom = ""
                currentUsername = ""
                for r in rooms:
                    if r[0] == conn:
                        currentRoom = r[1]
                        currentUsername = r[2]
                for r in rooms:
                    if r[1] == currentRoom:
                        if r[0] == conn:
                            print(f"[{addr}] {currentUsername} has been diconnected from room {currentRoom}.")
                            rooms.remove(r)
                counter = 0
                for r in rooms:
                    if r[1] == currentRoom:
                        counter = counter + 1
                if counter > 0:
                    print(f"[INFO] Sending to players in room {currentRoom} updated player list.")
                    messageToSend = ["setPlayers"]
                    for r in rooms:
                        if r[1] == currentRoom:
                            messageToSend.append(r[2])
                    for r in rooms:
                        if r[1] == currentRoom and r[0] != conn:
                            r[0].send(pickle.dumps(messageToSend))
                elif counter == 0:
                    for s in stackByRoom:
                        if s[0] == currentRoom:
                            stackByRoom.remove(s)
                    for h in heapByRoom:
                        if h[0] == currentRoom:
                            heapByRoom.remove(h)
                    for r in rooms:
                        if r[1] == currentRoom:
                            print(f"[{addr}] Room {currentRoom} deleted.")
                            rooms.remove(r)
                    for w in wayByRoom:
                        if w[0] == currentRoom:
                            wayByRoom.remove(w)
                            print(f"[{addr}] {currentUsername} was the last player in room {currentRoom}, stack and heap for this room has been deleted.")
                counterStack = False
                for s in stackByRoom:
                    if s[0] == currentRoom:
                        counterStack = True
                if counterStack == False:
                    for s in stackByRoom:
                        if s[0] == currentRoom:
                            stackByRoom.remove(s)
                    for h in heapByRoom:
                        if h[0] == currentRoom:
                            heapByRoom.remove(h)
                    for w in wayByRoom:
                        if w[0] == currentRoom:
                            wayByRoom.remove(w)
                            print(f"[{addr}] {currentUsername} disconnected room {currentRoom} that was playing. Stack and heap for this room has been deleted.")
                    roomToDelete = []
                    for r in rooms:
                        if r[1] == msg[2]:
                            roomToDelete.append(r)
                    for r in roomToDelete:
                        rooms.remove(r)
            elif msg[1] == "createRoom":
                randomRoomNum = random.choice(string.ascii_uppercase + string.digits) + random.choice(string.ascii_uppercase + string.digits) + random.choice(string.ascii_uppercase + string.digits) + random.choice(string.ascii_uppercase + string.digits) + random.choice(string.ascii_uppercase + string.digits)
                messageToSend = ['joinNow',f'{randomRoomNum}']
                rooms.append([conn, randomRoomNum, msg[0]])
                print(f"[{addr}] {msg[0]} created room number {randomRoomNum}")
                conn.send(pickle.dumps(messageToSend))
            elif msg[1] == "joinRoom":
                counter = 0
                for r in rooms:
                    if r[1] == msg[2]:
                        counter = counter + 1
                        if r[0] == conn:
                            print(f"[{addr}] {msg[0]} was already in room number {msg[2]}. He has been diconnected.")
                            rooms.remove(r)
                counterStack = False
                for s in stackByRoom:
                    if s[0] == msg[2]:
                        counterStack = True
                if counterStack == True:
                    messageToSend = ['error','roomCodeNotValid']
                    conn.send(pickle.dumps(messageToSend))
                    print(f"[{addr}] {msg[0]} tried to join room number {msg[2]}, but room doesn't exist or is full.")
                elif counter != 0 and counter < 4:
                    counterCheckUsername = 0
                    for r in rooms:
                        if r[1] == msg[2]:
                            if r[2] == msg[0]:
                                counterCheckUsername = counterCheckUsername + 1
                    if (counterCheckUsername == 0):
                        rooms.append([conn, msg[2], msg[0]])
                        print(f"[{addr}] {msg[0]} joined room number {msg[2]}. Sending update...")
                        messageToSend = ['joinNow',f'{msg[2]}']
                        conn.send(pickle.dumps(messageToSend))
                        messageToSend = ["setPlayers"]
                        for r in rooms:
                            if r[1] == msg[2]:
                                messageToSend.append(r[2])
                        for r in rooms:
                            if r[1] == msg[2] and r[0] != conn:
                                r[0].send(pickle.dumps(messageToSend))
                    else:
                        messageToSend = ['error','usernameNotValid']
                        conn.send(pickle.dumps(messageToSend))
                        print(f"[{addr}] {msg[0]} tried to join room number {msg[2]}, but username was already used.")
                else:
                    messageToSend = ['error','roomCodeNotValid']
                    conn.send(pickle.dumps(messageToSend))
                    print(f"[{addr}] {msg[0]} tried to join room number {msg[2]}, but room doesn't exist or is full.")
            elif msg[1] == "getPlayers":
                print(f"[INFO] {msg[0]} want players in room {msg[2]}.")
                messageToSend = ['setPlayers']
                for r in rooms:
                    if r[1] == msg[2]:
                        messageToSend.append(r[2])
                conn.send(pickle.dumps(messageToSend))
            elif msg[1] == "quitRoom":
                for r in rooms:
                    if r[1] == msg[2]:
                        if r[0] == conn:
                            print(f"[{addr}] {msg[0]} has been diconnected from room {msg[2]}.")
                            rooms.remove(r)
                counter = 0
                for r in rooms:
                    if r[1] == msg[2]:
                        counter = counter + 1
                if counter > 0:
                    print(f"[INFO] Sending to players in room {msg[2]} updated player list.")
                    messageToSend = ['setPlayers']
                    for r in rooms:
                        if r[1] == msg[2]:
                            messageToSend.append(r[2])
                    for r in rooms:
                        if r[1] == msg[2] and r[0] != conn:
                            r[0].send(pickle.dumps(messageToSend))
            elif msg[1] == "startGame":
                counter = 0
                for r in rooms:
                    if r[1] == msg[2]:
                        counter = counter + 1
                if counter == 0:
                    messageToSend = ['error','noRoomWithThisCode']
                    print(f"[INFO] {msg[0]} wanted to start room {msg[2]}, but there is no room with this code.")
                    conn.send(pickle.dumps(messageToSend))
                elif counter < 2:
                    messageToSend = ['error','notAValidNumberOfPerson']
                    print(f"[INFO] {msg[0]} wanted to start room {msg[2]}, but there wasn't enough players.")
                    conn.send(pickle.dumps(messageToSend))
                else:
                    tempStack = stack[:]
                    random.shuffle(tempStack)
                    currentRoom = ""
                    for r in rooms:
                        if r[0] == conn:
                            currentRoom = r[1]
                    for s in stackByRoom:
                        if s[0] == currentRoom:
                            stackByRoom.remove(s)
                    for h in heapByRoom:
                        if h[0] == currentRoom:
                            heapByRoom.remove(h)
                    for w in wayByRoom:
                        if w[0] == currentRoom:
                            wayByRoom.remove(w)
                    stackByRoom.append([msg[2],tempStack])
                    wayByRoom.append([msg[2],False])
                    for s in stackByRoom:
                        if s[0] == msg[2]:
                            roomPlaceInStack = stackByRoom.index(s)
                    counter = 0
                    hands = []
                    for r in rooms:
                        if r[1] == msg[2]:
                            counter = counter + 1
                    for c in range(0, counter):
                        tmpList = []
                        for i in range(0, 7):
                            tmpList.append(stackByRoom[roomPlaceInStack][1].pop(0))
                        hands.append(tmpList)
                    while ((stackByRoom[roomPlaceInStack][1][0][1] == 4) | (stackByRoom[roomPlaceInStack][1][0][0] in range(10, 13))) : 
                        random.shuffle(stackByRoom[roomPlaceInStack][1])
                    heapCard = stackByRoom[roomPlaceInStack][1].pop(0)
                    heapByRoom.append([msg[2],heapCard])
                    print(f"[INFO] {msg[0]} started room {msg[2]}.")
                    counter = 0
                    for r in rooms:
                        if r[1] == msg[2]:
                            messageToSend = ['startNow']
                            messageToSend.append(heapCard)
                            messageToSend.append(hands[counter])
                            if counter == 0:
                                messageToSend.append('yourTurn')
                            r[0].send(pickle.dumps(messageToSend))
                            counter = counter + 1
            elif msg[1] == "placing":
                print(f"[INFO] {msg[0]} placed card {msg[3]} in room {msg[2]}.")
                playersPlaying = []
                counter = -1
                indexCurrentPlayer = 0
                currentPlayerUsername = ""
                for r in rooms:
                    if r[1] == msg[2]:
                        playersPlaying.append(r)
                for w in wayByRoom:
                    if w[0] == msg[2]:
                        if w[1] == True:
                            playersPlaying.reverse()
                for p in playersPlaying:
                    counter = counter + 1
                    if p[2] == msg[0]:
                        indexCurrentPlayer = counter
                counterWayChanging = False
                for player in playersPlaying:
                    messageToSend = ['updateHeap']
                    messageToSend.append(msg[3])
                    messageToSend.append(playersPlaying[indexCurrentPlayer][2])
                    messageToSend.append(msg[4])
                    for heap in heapByRoom:
                        if heap[0] == msg[2]:
                            heap[1] = msg[3]
                    # check if card is "+2"
                    if msg[3][0] == 12 or (msg[3][1] == 4 and (msg[3][0] >= 6 and msg[3][0] <= 9)):
                        if indexCurrentPlayer+2 == len(playersPlaying):
                            messageToSend.append(playersPlaying[0][2])
                            if playersPlaying.index(player) == 0:
                                messageToSend.append('yourTurn')
                        elif indexCurrentPlayer+2 > len(playersPlaying):
                            messageToSend.append(playersPlaying[1][2])
                            if playersPlaying.index(player) == 1:
                                messageToSend.append('yourTurn')
                        elif playersPlaying.index(player) == indexCurrentPlayer+2:
                            messageToSend.append(playersPlaying[indexCurrentPlayer+2][2])
                            messageToSend.append('yourTurn')
                        else:
                            messageToSend.append(playersPlaying[indexCurrentPlayer+2][2])
                        if indexCurrentPlayer+1 == len(playersPlaying):
                            if playersPlaying.index(player) == 0:
                                for s in stackByRoom:
                                    if s[0] == msg[2]:
                                        roomPlaceInStack = stackByRoom.index(s)
                                if len(stackByRoom[roomPlaceInStack][1]) <=2:
                                    tempStack = stack[:]
                                    random.shuffle(tempStack)
                                    stackByRoom[roomPlaceInStack][1] = tempStack
                                if msg[3][0] == 12:
                                    messageToSend.append([stackByRoom[roomPlaceInStack][1].pop(0),stackByRoom[roomPlaceInStack][1].pop(0)])
                                    messageToSend.append('takeCards')
                                else:
                                    messageToSend.append([stackByRoom[roomPlaceInStack][1].pop(0),stackByRoom[roomPlaceInStack][1].pop(0),stackByRoom[roomPlaceInStack][1].pop(0),stackByRoom[roomPlaceInStack][1].pop(0)])
                                    messageToSend.append('takeCards')
                        elif playersPlaying.index(player) == indexCurrentPlayer+1:
                            for s in stackByRoom:
                                if s[0] == msg[2]:
                                    roomPlaceInStack = stackByRoom.index(s)
                            if len(stackByRoom[roomPlaceInStack][1]) <= 4:
                                tempStack = stack[:]
                                random.shuffle(tempStack)
                                stackByRoom[roomPlaceInStack][1] = tempStack
                            if msg[3][0] == 12:
                                messageToSend.append([stackByRoom[roomPlaceInStack][1].pop(0),stackByRoom[roomPlaceInStack][1].pop(0)])
                                messageToSend.append('takeCards')
                            else:
                                messageToSend.append([stackByRoom[roomPlaceInStack][1].pop(0),stackByRoom[roomPlaceInStack][1].pop(0),stackByRoom[roomPlaceInStack][1].pop(0),stackByRoom[roomPlaceInStack][1].pop(0)])
                                messageToSend.append('takeCards')
                    # check if card is "interdiction de jouer"
                    elif msg[3][0] == 10:
                        if indexCurrentPlayer+2 == len(playersPlaying):
                            messageToSend.append(playersPlaying[0][2])
                            if playersPlaying.index(player) == 0:
                                messageToSend.append('yourTurn')
                        elif indexCurrentPlayer+2 > len(playersPlaying):
                            messageToSend.append(playersPlaying[1][2])
                            if playersPlaying.index(player) == 1:
                                messageToSend.append('yourTurn')
                        elif playersPlaying.index(player) == indexCurrentPlayer+2:
                            messageToSend.append(playersPlaying[indexCurrentPlayer+2][2])
                            messageToSend.append('yourTurn')
                        else:
                            messageToSend.append(playersPlaying[indexCurrentPlayer+2][2])
                    # Changement de sens
                    elif msg[3][0] == 11:
                        for w in wayByRoom:
                            if w[0] == msg[2] and counterWayChanging == False:
                                if w[1] == False:
                                    w[1] = True
                                    counterWayChanging = True
                                    break
                                else:
                                    w[1] = False
                                    counterWayChanging = True
                                    break
                        if indexCurrentPlayer-1 < 0:
                            messageToSend.append(playersPlaying[len(playersPlaying)-1][2])
                            if playersPlaying.index(player) == len(playersPlaying)-1:
                                messageToSend.append('yourTurn')
                        elif playersPlaying.index(player) == indexCurrentPlayer-1:
                            messageToSend.append(playersPlaying[indexCurrentPlayer-1][2])
                            messageToSend.append('yourTurn')
                        else:
                            messageToSend.append(playersPlaying[indexCurrentPlayer-1][2])
                    # Else, normal skip
                    else:
                        if indexCurrentPlayer+1 == len(playersPlaying):
                            messageToSend.append(playersPlaying[0][2])
                            if playersPlaying.index(player) == 0:
                                messageToSend.append('yourTurn')
                        elif playersPlaying.index(player) == indexCurrentPlayer+1:
                            messageToSend.append(playersPlaying[indexCurrentPlayer+1][2])
                            messageToSend.append('yourTurn')
                        else:
                            messageToSend.append(playersPlaying[indexCurrentPlayer+1][2])
                    player[0].send(pickle.dumps(messageToSend))
            elif msg[1] == "demandingCard":
                print(f"[INFO] {msg[0]} ask for a card in room {msg[2]}.")
                playersPlaying = []
                counter = -1
                indexCurrentPlayer = 0
                for r in rooms:
                    if r[1] == msg[2]:
                        playersPlaying.append(r)
                for w in wayByRoom:
                    if w[0] == msg[2]:
                        if w[1] == True:
                            print("reversing")
                            playersPlaying.reverse()
                for p in playersPlaying:
                    counter = counter + 1
                    if p[2] == msg[0]:
                        indexCurrentPlayer = counter
                for player in playersPlaying:
                    messageToSend = ['askedCard']
                    messageToSend.append(playersPlaying[indexCurrentPlayer][2])
                    messageToSend.append(msg[3])
                    if indexCurrentPlayer+1 == len(playersPlaying):
                        messageToSend.append(playersPlaying[0][2])
                        if playersPlaying.index(player) == 0:
                            messageToSend.append('yourTurn')
                    elif playersPlaying.index(player) == indexCurrentPlayer+1:
                        messageToSend.append(playersPlaying[indexCurrentPlayer+1][2])
                        messageToSend.append('yourTurn')
                    else:
                        messageToSend.append(playersPlaying[indexCurrentPlayer+1][2])
                    if player[0] != conn:
                        player[0].send(pickle.dumps(messageToSend))
                    else:
                        messageToSend = ['newCard']
                        messageToSend.append(playersPlaying[indexCurrentPlayer][2])
                        messageToSend.append(msg[3])
                        for s in stackByRoom:
                            if s[0] == msg[2]:
                                roomPlaceInStack = stackByRoom.index(s)
                        if len(stackByRoom[roomPlaceInStack][1]) <= 1:
                            tempStack = stack[:]
                            random.shuffle(tempStack)
                            stackByRoom[roomPlaceInStack][1] = tempStack
                        messageToSend.append(stackByRoom[roomPlaceInStack][1].pop(0))
                        if indexCurrentPlayer+1 == len(playersPlaying):
                            messageToSend.append(playersPlaying[0][2])
                        else:
                            messageToSend.append(playersPlaying[indexCurrentPlayer+1][2])
                        conn.send(pickle.dumps(messageToSend))
            elif msg[1] == "updateNumberOfCards":
                print(f"[INFO] {msg[0]} sent update of his hand in room {msg[2]}.")
                messageToSend = ['updateNumberOfCards']
                messageToSend.append(msg[0])
                messageToSend.append(msg[3])
                for r in rooms:
                    if r[1] == msg[2]:
                        r[0].send(pickle.dumps(messageToSend))
            elif msg[1] == "uno":
                print(f"[INFO] {msg[0]} sent uno message in room {msg[2]}.")
                messageToSend = ['unoFromPlayer']
                messageToSend.append(msg[0])
                for r in rooms:
                    if r[1] == msg[2]:
                        r[0].send(pickle.dumps(messageToSend))
            elif msg[1] == "counterUno":
                print(f"[INFO] {msg[0]} sent counter uno message in room {msg[2]}.")
                messageToSend = ['takeCardsCounterUnoHit']
                for s in stackByRoom:
                    if s[0] == msg[2]:
                        roomPlaceInStack = stackByRoom.index(s)
                if len(stackByRoom[roomPlaceInStack][1]) <=2:
                    tempStack = stack[:]
                    random.shuffle(tempStack)
                    stackByRoom[roomPlaceInStack][1] = tempStack
                messageToSend.append([stackByRoom[roomPlaceInStack][1].pop(0),stackByRoom[roomPlaceInStack][1].pop(0)])
                for r in rooms:
                    if r[1] == msg[2] and r[2] == msg[3]:
                        r[0].send(pickle.dumps(messageToSend))
            elif msg[1] == "takeCardsCounterUno":
                print(f"[INFO] {msg[0]} has to take card due to bad counter uno in room {msg[2]}.")
                messageToSend = ['takeCardsCounterUnoHit']
                for s in stackByRoom:
                    if s[0] == msg[2]:
                        roomPlaceInStack = stackByRoom.index(s)
                if len(stackByRoom[roomPlaceInStack][1]) <=2:
                    tempStack = stack[:]
                    random.shuffle(tempStack)
                    stackByRoom[roomPlaceInStack][1] = tempStack
                messageToSend.append([stackByRoom[roomPlaceInStack][1].pop(0),stackByRoom[roomPlaceInStack][1].pop(0)])
                for r in rooms:
                    if r[1] == msg[2] and r[2] == msg[0]:
                        r[0].send(pickle.dumps(messageToSend))
            elif msg[1] == "winning":
                print(f"[INFO] {msg[0]} won the room {msg[2]}.")
                for r in rooms:
                    print("sending loop")
                    if r[1] == msg[2]:
                        print(f"sending to {r[2]}")
                        messageToSend = ['partyEnd']
                        messageToSend.append(msg[0])
                        r[0].send(pickle.dumps(messageToSend))
            elif msg[1] == "leaveRoom":
                print(f"[INFO] {msg[0]} left the room {msg[2]}.")
                for r in rooms:
                    if r[1] == msg[2]:
                        messageToSend = ['setPlayers']
                        messageToSend.append(msg[0])
                        r[0].send(pickle.dumps(messageToSend))
                currentRoom = ""
                for r in rooms:
                    if r[0] == conn:
                        currentRoom = r[1]
                for s in stackByRoom:
                    if s[0] == currentRoom:
                        stackByRoom.remove(s)
                for h in heapByRoom:
                    if h[0] == currentRoom:
                        heapByRoom.remove(h)
                for w in wayByRoom:
                    if w[0] == currentRoom:
                        wayByRoom.remove(w)
                roomToDelete = []
                for r in rooms:
                    if r[1] == msg[2]:
                        roomToDelete.append(r)
                for r in roomToDelete:
                    rooms.remove(r)
                
                #conn.send(msg.encode(FORMAT))
                # Pour envoyer a tous les clients
                #for connClient in connections:
                    #connClient.send(msg.encode(FORMAT))
    conn.close()
    connections.remove(conn)

In [4]:
def start():
    server.listen()
    print(f"[LISTENING] Server is listening on {SERVER}")
    while True:
        conn, addr = server.accept()
        thread = threading.Thread(target=handle_client, args=(conn, addr))
        thread.start()
        print(f"[ACTIVE CONNECTION] {threading.active_count() - 8 }")
print("[STARTING] Sever is starting…")

[STARTING] Sever is starting…


In [ ]:
start()

[LISTENING] Server is listening on 0.0.0.0
